In [1]:
import pandas as pd
import numpy as np
from scipy import stats

C:\Users\qf3636ll\AppData\Local\Temp\ipykernel_23588\1521899416.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


FUNCTIONS LIBRARY

In [2]:
def encode_GPA(df):
    for row in df.index:
        if df.loc[row,'GRADE'] == 'A':
            df.loc[row,'GRADE2'] = 4.0
        elif df.loc[row,'GRADE'] == 'B':
            df.loc[row,'GRADE2'] = 3.0
        elif df.loc[row,'GRADE'] == 'C':
            df.loc[row,'GRADE2'] = 2.0
        elif df.loc[row,'GRADE'] == 'W':
            df.loc[row,'GRADE2'] = 0.0
        elif df.loc[row,'GRADE'] == 'F':
            df.loc[row,'GRADE2'] = 0.0
        elif df.loc[row,'GRADE'] == 'D':
            df.loc[row,'GRADE2'] = 1.0
        elif df.loc[row,'GRADE'] == 'FW':
            df.loc[row,'GRADE2'] = 0.0
        elif df.loc[row,'GRADE'] == 'P':
            df.loc[row,'GRADE2'] = 4.0
        elif df.loc[row,'GRADE'] == 'I':
            df.loc[row,'GRADE2'] = 0.0
        elif df.loc[row,'GRADE'] == 'NC':
            df.loc[row,'GRADE2'] = 0.0
    return df

def encode_SuccessRate(df):
    for row in df.index:
        if df.loc[row,'COURSE_SUCCESS'] == 'Course Success':
            df.loc[row,'CS'] = 1.0
        else:
            df.loc[row,'CS'] = 0.0
    return df

def encode_GPApnts(df):
    df['GPApnts'] = df['GRADE2'] * df['REG_CRHR']
    return df

def calc_GPA2(df):
    HC = df[['TECH_ID']].drop_duplicates(keep='first').reset_index(drop=True)
    out = pd.DataFrame(columns=
        ['TECH_ID','GPA','Credits','Courses'])
    stu, gp, gpa, creds, cou = 0,0,0,0,0
    termc, termgp, termgpa, termcou = 0,0,0,0
    for i in HC['TECH_ID'].values:
        stu = i
        creds = df.loc[df['TECH_ID'] == i,'REG_CRHR'].sum()
        cou = df.loc[df['TECH_ID'] == i,'REG_CRHR'].count()
        gp = df.loc[df['TECH_ID'] == i,'GPApnts'].sum()
        gpa = gp / creds
        cs = df.loc[df['TECH_ID'] == i,'CS'].sum()
        sr = cs / cou
        newrow = pd.DataFrame({
            'TECH_ID': stu,
            'GPA': gpa,
            'Credits': creds,
            'Courses': cou,
            'SR': sr},
            index = [1])
        out = pd.concat([out,newrow])
    return out

In [3]:
##retrieve data
LLs = pd.read_csv('LLab_registrations.csv')
others = pd.read_csv('other_courseRecords.csv')
LLcR = pd.read_csv('LLab_courseRecords.csv')
LLnS = pd.read_csv('LLab_nextSemester.csv')
AS = pd.read_csv('AcadSusp.csv')
majors = pd.read_csv('AllStudentMajors.csv')

In [4]:
##convert letter grades to GPs
encode_SuccessRate(encode_GPApnts(encode_GPA(LLcR)))
encode_SuccessRate(encode_GPApnts(encode_GPA(others)))
##create a GPA with just the semester GPAs for all students
LLcR_gpa = calc_GPA2(LLcR)
others_gpa = calc_GPA2(others)

C:\Users\qf3636ll\AppData\Local\Temp\ipykernel_23588\1180483069.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out = pd.concat([out,newrow])
C:\Users\qf3636ll\AppData\Local\Temp\ipykernel_23588\1180483069.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out = pd.concat([out,newrow])


In [5]:
registrations = LLs[['TECH_ID','YRTR','COU_ID','DROP_TIME_STAMP','GRADE']]
LL_majors = pd.merge(registrations, majors, how='left', on='TECH_ID')
labbers = LL_majors.loc[(LL_majors['DROP_TIME_STAMP'].isna() == True) & (LL_majors['GRADE'].isna() == True),:]
drops = LL_majors.loc[(LL_majors['DROP_TIME_STAMP'].isna() == False) | (LL_majors['GRADE'].isna() == False),:]

In [6]:
#merge students with their majors
LLcR_majors1 = pd.merge(LLcR, majors, how='left', on='TECH_ID')
LLcR_majors = pd.merge(LLcR_majors1, others_gpa, how='left', on='TECH_ID')
#merge student course records and majors
cR_majors1 = pd.merge(others, majors, how='left', on='TECH_ID')
cR_majors2 = cR_majors1[['TECH_ID','ADM_STAT','AGE_CATEGORY','STUDENT_CLASSIFICATION','NEW_CONTINUING_STUDENT','STUDENT_LOAD', 'STUDENT_GENDER','UNDERREPRESENTED_DESC','FIRST_GEN_FED_DESC','FIRST_GEN_MN_DESC','PELL_RECPT_DESC','STUDENT_OF_COLOR_DESC','REPORTING_RACE_ETHNICITY','MAJOR_ID_1','MAJOR_DEGREE_CODE_1','MAJOR_DESC_1']].copy()
cR_majors2.drop_duplicates(inplace=True, ignore_index=True)
cR_majors = pd.merge(cR_majors2, others_gpa, how='left', on='TECH_ID')
cR_majors['credits2'] = cR_majors['SR'] * cR_majors['Credits']

In [ ]:
out = pd.DataFrame(columns=
            ['TECH_ID','GPA','SR','Credits',
            't1_GPA','t1_SR','t1_str','t1_count','t2_GPA','t2_SR','t2_str','t2_count',
            't3_GPA','t3_SR','t3_str','t3_count','t4_GPA','t4_SR','t4_str','t4_count',
            't5_GPA','t5_SR','t5_str','t5_count','t6_GPA','t6_SR','t6_str','t6_count',
            't7_GPA','t7_SR','t7_str','t7_count','t8_GPA','t8_SR','t8_str','t8_count',
            't9_GPA','t9_SR','t9_str','t9_count','t10_GPA','t10_SR','t10_str','t10_count'])
listy1 = labbers['TECH_ID'].drop_duplicates().to_list()
t1,t2,t3,t4,t5,t6,t7,t8,t9,t10 = 0,0,0,0,0,0,0,0,0,0
for i in listy1:
    stu = i
    stu_df = LLcR_majors.loc[LLcR_majors['TECH_ID'] == stu, :]
    stu_df.reset_index(inplace=True, drop=True)
    dem_df_f = stu_df[['ADM_STAT','AGE_CATEGORY','STUDENT_CLASSIFICATION','NEW_CONTINUING_STUDENT','STUDENT_LOAD',
                       'STUDENT_GENDER','UNDERREPRESENTED_DESC','FIRST_GEN_FED_DESC','FIRST_GEN_MN_DESC','PELL_RECPT_DESC',
                       'STUDENT_OF_COLOR_DESC','REPORTING_RACE_ETHNICITY','MAJOR_ID_1','MAJOR_DEGREE_CODE_1','MAJOR_DESC_1',
                       'GPA','Credits','SR']]
    dem_df = dem_df_f.copy().drop_duplicates()

    adm = dem_df.iloc[0,0]
    ac = dem_df.iloc[0,1]
    #ag = dem_df.iloc[0,2]
    sc = dem_df.iloc[0,2]
    ncs = dem_df.iloc[0,3]
    sl = dem_df.iloc[0,4]
    sg = dem_df.iloc[0,5]
    ud = dem_df.iloc[0,6]
    fgf = dem_df.iloc[0,7]
    fgm = dem_df.iloc[0,8]
    #pe = dem_df.iloc[0,10]
    pr = dem_df.iloc[0,9]
    soc = dem_df.iloc[0,10]
    rre = dem_df.iloc[0,11]
    mid = dem_df.iloc[0,12]
    mdc = dem_df.iloc[0,13]
    mn = dem_df.iloc[0,14]
    GPA = dem_df.iloc[0,15]
    STR = dem_df.iloc[0,16]
    SR = dem_df.iloc[0,17]
    #general demographics window; no first gen MN, no reporting race ethnicity, 
    test1 = cR_majors.loc[
        (cR_majors['ADM_STAT']==adm) & (cR_majors['STUDENT_CLASSIFICATION']==sc) & (cR_majors['STUDENT_LOAD']==sl)
        ,['TECH_ID','GPA','SR','Credits']]
    t1_GPA = GPA - test1['GPA'].mean()
    t1_SR = SR - test1['SR'].mean()
    t1_str = STR - test1['Credits'].mean()
    t1_count = test1['TECH_ID'].count()
    test2 = cR_majors.loc[
        (cR_majors['ADM_STAT']==adm) & (cR_majors['AGE_CATEGORY']==ac) & (cR_majors['STUDENT_CLASSIFICATION']==sc) & (cR_majors['STUDENT_LOAD']==sl)
        ,['TECH_ID','GPA','SR','Credits']]
    t2_GPA = GPA - test2['GPA'].mean()
    t2_SR = SR - test2['SR'].mean()
    t2_str = STR - test2['Credits'].mean()
    t2_count = test2['TECH_ID'].count()
    test3 = cR_majors.loc[
        (cR_majors['ADM_STAT']==adm) & (cR_majors['STUDENT_LOAD']==sl) &
        (cR_majors['FIRST_GEN_FED_DESC']==fgf) & (cR_majors['PELL_RECPT_DESC']==pr) & (cR_majors['STUDENT_OF_COLOR_DESC']==soc)
        ,['TECH_ID','GPA','SR','Credits']]
    t3_GPA = GPA - test3['GPA'].mean()
    t3_SR = SR - test3['SR'].mean()
    t3_str = STR - test3['Credits'].mean()
    t3_count = test3['TECH_ID'].count()
    test4 = cR_majors.loc[
        (cR_majors['ADM_STAT']==adm) & (cR_majors['AGE_CATEGORY']==ac) & (cR_majors['STUDENT_LOAD']==sl) &
        (cR_majors['FIRST_GEN_FED_DESC']==fgf) & (cR_majors['PELL_RECPT_DESC']==pr) & (cR_majors['STUDENT_OF_COLOR_DESC']==soc)
        ,['TECH_ID','GPA','SR','Credits']]
    t4_GPA = GPA - test4['GPA'].mean()
    t4_SR = SR - test4['SR'].mean()
    t4_str = STR - test4['Credits'].mean()
    t4_count = test4['TECH_ID'].count()
    #narrow demographics window2
    test5 = cR_majors.loc[
        (cR_majors['ADM_STAT']==adm) & (cR_majors['STUDENT_LOAD']==sl) &
        (cR_majors['STUDENT_GENDER']==sg) & (cR_majors['FIRST_GEN_FED_DESC']==fgf) & (cR_majors['FIRST_GEN_MN_DESC']==fgm) & 
        (cR_majors['PELL_RECPT_DESC']==pr) & (cR_majors['STUDENT_OF_COLOR_DESC']==soc)
        ,['TECH_ID','GPA','SR','Credits']]
    t5_GPA = GPA - test5['GPA'].mean()
    t5_SR = SR - test5['SR'].mean()
    t5_str = STR - test5['Credits'].mean()
    t5_count = test5['TECH_ID'].count()
    #narrow demographics window3
    test6 = cR_majors.loc[
        (cR_majors['ADM_STAT']==adm) & (cR_majors['AGE_CATEGORY']==ac) & (cR_majors['STUDENT_CLASSIFICATION']==sc) & (cR_majors['STUDENT_LOAD']==sl) &
        (cR_majors['FIRST_GEN_FED_DESC']==fgf) & (cR_majors['FIRST_GEN_MN_DESC']==fgm) & (cR_majors['PELL_RECPT_DESC']==pr) & (cR_majors['STUDENT_OF_COLOR_DESC']==soc)
        ,['TECH_ID','GPA','SR','Credits']]
    t6_GPA = GPA - test6['GPA'].mean()
    t6_SR = SR - test6['SR'].mean()
    t6_str = STR - test6['Credits'].mean()
    t6_count = test6['TECH_ID'].count()
    #narrow demographics window4
    test7 = cR_majors.loc[
        (cR_majors['ADM_STAT']==adm) & (cR_majors['STUDENT_CLASSIFICATION']==sc) & (cR_majors['STUDENT_LOAD']==sl) &
        (cR_majors['FIRST_GEN_FED_DESC']==fgf) & (cR_majors['FIRST_GEN_MN_DESC']==fgm) & (cR_majors['PELL_RECPT_DESC']==pr) & (cR_majors['REPORTING_RACE_ETHNICITY']==rre)
        ,['TECH_ID','GPA','SR','Credits']]
    t7_GPA = GPA - test7['GPA'].mean()
    t7_SR = SR - test7['SR'].mean()
    t7_str = STR - test7['Credits'].mean()
    t7_count = test7['TECH_ID'].count()
    #just others in their major
    test8 = cR_majors.loc[ (cR_majors['MAJOR_ID_1']==mid) & (cR_majors['MAJOR_DEGREE_CODE_1']==mdc) & (cR_majors['MAJOR_DESC_1']==mn),['TECH_ID','GPA','SR','Credits']]
    t8_GPA = GPA - test8['GPA'].mean()
    t8_SR = SR - test8['SR'].mean()
    t8_str = STR - test8['Credits'].mean()
    t8_count = test8['TECH_ID'].count()
    #major and general demographics
    test9 = cR_majors.loc[ (cR_majors['MAJOR_ID_1']==mid) & (cR_majors['MAJOR_DEGREE_CODE_1']==mdc) & (cR_majors['MAJOR_DESC_1']==mn) &
        (cR_majors['ADM_STAT']==adm) & (cR_majors['STUDENT_CLASSIFICATION']==sc) & (cR_majors['STUDENT_LOAD']==sl)
        ,['TECH_ID','GPA','SR','Credits']]
    t9_GPA = GPA - test9['GPA'].mean()
    t9_SR = SR - test9['SR'].mean()
    t9_str = STR - test9['Credits'].mean()
    t9_count = test9['TECH_ID'].count()
    #major and mid demographics
    test10 = cR_majors.loc[ (cR_majors['MAJOR_ID_1']==mid) & (cR_majors['MAJOR_DEGREE_CODE_1']==mdc) & (cR_majors['MAJOR_DESC_1']==mn) &
        (cR_majors['ADM_STAT']==adm) & (cR_majors['STUDENT_CLASSIFICATION']==sc) & (cR_majors['STUDENT_LOAD']==sl) &
        (cR_majors['FIRST_GEN_FED_DESC']==fgf) & (cR_majors['PELL_RECPT_DESC']==pr) & (cR_majors['STUDENT_OF_COLOR_DESC']==soc)
        ,['TECH_ID','GPA','SR','Credits']]
    t10_GPA = GPA - test10['GPA'].mean()
    t10_SR = SR - test10['SR'].mean()
    t10_str = STR - test10['Credits'].mean()
    t10_count = test10['TECH_ID'].count()
    #row for all of the student's test results
    stu_pid = pd.DataFrame(
                            {'TECH_ID':stu,'GPA':GPA,'SR':SR,'Credits':STR,'STUDENT_LOAD': sl,'REPORTING_RACE_ETHNICITY':rre,
                            't1_GPA':t1_GPA,'t1_SR':t1_SR,'t1_str':t1_str,'t1_count':t1_count,
                            't2_GPA':t2_GPA,'t2_SR':t2_SR,'t2_str':t2_str,'t2_count':t2_count,
                            't3_GPA':t3_GPA,'t3_SR':t3_SR,'t3_str':t3_str,'t3_count':t3_count,
                            't4_GPA':t4_GPA,'t4_SR':t4_SR,'t4_str':t4_str,'t4_count':t4_count,
                            't5_GPA':t5_GPA,'t5_SR':t5_SR,'t5_str':t5_str,'t5_count':t5_count,
                            't6_GPA':t6_GPA,'t6_SR':t6_SR,'t6_str':t6_str,'t6_count':t6_count,
                            't7_GPA':t7_GPA,'t7_SR':t7_SR,'t7_str':t7_str,'t7_count':t7_count,
                            't8_GPA':t8_GPA,'t8_SR':t8_SR,'t8_str':t8_str,'t8_count':t8_count,
                            't9_GPA':t9_GPA,'t9_SR':t9_SR,'t9_str':t9_str,'t9_count':t9_count,
                            't10_GPA':t10_GPA,'t10_SR':t10_SR,'t10_str':t10_str,'t10_count':t10_count}, index=[1])
    out = pd.concat([stu_pid,out])
        

In [7]:
##Version2
##instead of the outputs being the difference, the outputs are the mean success and gpa of the students in the comparison groups
out2 = pd.DataFrame(columns=
            ['TECH_ID','GPA','SR','Credits',
            't1_GPA','t1_SR','t1_str','t1_count','t2_GPA','t2_SR','t2_str','t2_count',
            't3_GPA','t3_SR','t3_str','t3_count','t4_GPA','t4_SR','t4_str','t4_count',
            't5_GPA','t5_SR','t5_str','t5_count','t6_GPA','t6_SR','t6_str','t6_count',
            't7_GPA','t7_SR','t7_str','t7_count','t8_GPA','t8_SR','t8_str','t8_count',
            't9_GPA','t9_SR','t9_str','t9_count','t10_GPA','t10_SR','t10_str','t10_count'])
listy1 = labbers['TECH_ID'].drop_duplicates().to_list()
t1,t2,t3,t4,t5,t6,t7,t8,t9,t10 = 0,0,0,0,0,0,0,0,0,0
for i in listy1:
    stu = i
    stu_df = LLcR_majors.loc[LLcR_majors['TECH_ID'] == stu, :]
    stu_df.reset_index(inplace=True, drop=True)
    dem_df_f = stu_df[['ADM_STAT','AGE_CATEGORY','STUDENT_CLASSIFICATION','NEW_CONTINUING_STUDENT','STUDENT_LOAD',
                       'STUDENT_GENDER','UNDERREPRESENTED_DESC','FIRST_GEN_FED_DESC','FIRST_GEN_MN_DESC','PELL_RECPT_DESC',
                       'STUDENT_OF_COLOR_DESC','REPORTING_RACE_ETHNICITY','MAJOR_ID_1','MAJOR_DEGREE_CODE_1','MAJOR_DESC_1',
                       'GPA','Credits','SR']]
    dem_df = dem_df_f.copy().drop_duplicates()

    adm = dem_df.iloc[0,0]
    ac = dem_df.iloc[0,1]
    #ag = dem_df.iloc[0,2]
    sc = dem_df.iloc[0,2]
    ncs = dem_df.iloc[0,3]
    sl = dem_df.iloc[0,4]
    sg = dem_df.iloc[0,5]
    ud = dem_df.iloc[0,6]
    fgf = dem_df.iloc[0,7]
    fgm = dem_df.iloc[0,8]
    #pe = dem_df.iloc[0,10]
    pr = dem_df.iloc[0,9]
    soc = dem_df.iloc[0,10]
    rre = dem_df.iloc[0,11]
    mid = dem_df.iloc[0,12]
    mdc = dem_df.iloc[0,13]
    mn = dem_df.iloc[0,14]
    GPA = dem_df.iloc[0,15]
    STR = dem_df.iloc[0,16]
    SR = dem_df.iloc[0,17]
    #general demographics window; no first gen MN, no reporting race ethnicity, 
    test1 = cR_majors.loc[
        (cR_majors['ADM_STAT']==adm) & (cR_majors['STUDENT_CLASSIFICATION']==sc) & (cR_majors['STUDENT_LOAD']==sl)
        ,['TECH_ID','GPA','SR','Credits']]
    t1_GPA = test1['GPA'].mean()
    t1_SR = test1['SR'].mean()
    t1_str = test1['Credits'].mean()
    t1_count = test1['TECH_ID'].count()
    test2 = cR_majors.loc[
        (cR_majors['ADM_STAT']==adm) & (cR_majors['AGE_CATEGORY']==ac) & (cR_majors['STUDENT_CLASSIFICATION']==sc) & (cR_majors['STUDENT_LOAD']==sl)
        ,['TECH_ID','GPA','SR','Credits']]
    t2_GPA = test2['GPA'].mean()
    t2_SR = test2['SR'].mean()
    t2_str = test2['Credits'].mean()
    t2_count = test2['TECH_ID'].count()
    test3 = cR_majors.loc[
        (cR_majors['ADM_STAT']==adm) & (cR_majors['STUDENT_LOAD']==sl) &
        (cR_majors['FIRST_GEN_FED_DESC']==fgf) & (cR_majors['PELL_RECPT_DESC']==pr) & (cR_majors['STUDENT_OF_COLOR_DESC']==soc)
        ,['TECH_ID','GPA','SR','Credits']]
    t3_GPA = test3['GPA'].mean()
    t3_SR = test3['SR'].mean()
    t3_str = test3['Credits'].mean()
    t3_count = test3['TECH_ID'].count()
    test4 = cR_majors.loc[
        (cR_majors['ADM_STAT']==adm) & (cR_majors['AGE_CATEGORY']==ac) & (cR_majors['STUDENT_LOAD']==sl) &
        (cR_majors['FIRST_GEN_FED_DESC']==fgf) & (cR_majors['PELL_RECPT_DESC']==pr) & (cR_majors['STUDENT_OF_COLOR_DESC']==soc)
        ,['TECH_ID','GPA','SR','Credits']]
    t4_GPA = test4['GPA'].mean()
    t4_SR = test4['SR'].mean()
    t4_str = test4['Credits'].mean()
    t4_count = test4['TECH_ID'].count()
    #narrow demographics window2
    test5 = cR_majors.loc[
        (cR_majors['ADM_STAT']==adm) & (cR_majors['STUDENT_LOAD']==sl) &
        (cR_majors['STUDENT_GENDER']==sg) & (cR_majors['FIRST_GEN_FED_DESC']==fgf) & (cR_majors['FIRST_GEN_MN_DESC']==fgm) & 
        (cR_majors['PELL_RECPT_DESC']==pr) & (cR_majors['STUDENT_OF_COLOR_DESC']==soc)
        ,['TECH_ID','GPA','SR','Credits']]
    t5_GPA = test5['GPA'].mean()
    t5_SR = test5['SR'].mean()
    t5_str = test5['Credits'].mean()
    t5_count = test5['TECH_ID'].count()
    #narrow demographics window3
    test6 = cR_majors.loc[
        (cR_majors['ADM_STAT']==adm) & (cR_majors['AGE_CATEGORY']==ac) & (cR_majors['STUDENT_CLASSIFICATION']==sc) & (cR_majors['STUDENT_LOAD']==sl) &
        (cR_majors['FIRST_GEN_FED_DESC']==fgf) & (cR_majors['FIRST_GEN_MN_DESC']==fgm) & (cR_majors['PELL_RECPT_DESC']==pr) & (cR_majors['STUDENT_OF_COLOR_DESC']==soc)
        ,['TECH_ID','GPA','SR','Credits']]
    t6_GPA = test6['GPA'].mean()
    t6_SR = test6['SR'].mean()
    t6_str = test6['Credits'].mean()
    t6_count = test6['TECH_ID'].count()
    #narrow demographics window4
    test7 = cR_majors.loc[
        (cR_majors['ADM_STAT']==adm) & (cR_majors['STUDENT_CLASSIFICATION']==sc) & (cR_majors['STUDENT_LOAD']==sl) &
        (cR_majors['FIRST_GEN_FED_DESC']==fgf) & (cR_majors['FIRST_GEN_MN_DESC']==fgm) & (cR_majors['PELL_RECPT_DESC']==pr) & (cR_majors['REPORTING_RACE_ETHNICITY']==rre)
        ,['TECH_ID','GPA','SR','Credits']]
    t7_GPA = test7['GPA'].mean()
    t7_SR = test7['SR'].mean()
    t7_str = test7['Credits'].mean()
    t7_count = test7['TECH_ID'].count()
    #just others in their major
    test8 = cR_majors.loc[ (cR_majors['MAJOR_ID_1']==mid) & (cR_majors['MAJOR_DEGREE_CODE_1']==mdc) & (cR_majors['MAJOR_DESC_1']==mn),['TECH_ID','GPA','SR','Credits']]
    t8_GPA = test8['GPA'].mean()
    t8_SR = test8['SR'].mean()
    t8_str = test8['Credits'].mean()
    t8_count = test8['TECH_ID'].count()
    #major and general demographics
    test9 = cR_majors.loc[ (cR_majors['MAJOR_ID_1']==mid) & (cR_majors['MAJOR_DEGREE_CODE_1']==mdc) & (cR_majors['MAJOR_DESC_1']==mn) &
        (cR_majors['ADM_STAT']==adm) & (cR_majors['STUDENT_CLASSIFICATION']==sc) & (cR_majors['STUDENT_LOAD']==sl)
        ,['TECH_ID','GPA','SR','Credits']]
    t9_GPA = test9['GPA'].mean()
    t9_SR = test9['SR'].mean()
    t9_str = test9['Credits'].mean()
    t9_count = test9['TECH_ID'].count()
    #major and mid demographics
    test10 = cR_majors.loc[ (cR_majors['MAJOR_ID_1']==mid) & (cR_majors['MAJOR_DEGREE_CODE_1']==mdc) & (cR_majors['MAJOR_DESC_1']==mn) &
        (cR_majors['ADM_STAT']==adm) & (cR_majors['STUDENT_CLASSIFICATION']==sc) & (cR_majors['STUDENT_LOAD']==sl) &
        (cR_majors['FIRST_GEN_FED_DESC']==fgf) & (cR_majors['PELL_RECPT_DESC']==pr) & (cR_majors['STUDENT_OF_COLOR_DESC']==soc)
        ,['TECH_ID','GPA','SR','Credits']]
    t10_GPA = test10['GPA'].mean()
    t10_SR = test10['SR'].mean()
    t10_str = test10['Credits'].mean()
    t10_count = test10['TECH_ID'].count()
    #row for all of the student's test results
    stu_pid = pd.DataFrame(
                            {'TECH_ID':stu,'GPA':GPA,'SR':SR,'Credits':STR,'STUDENT_LOAD': sl,'REPORTING_RACE_ETHNICITY':rre,
                            't1_GPA':t1_GPA,'t1_SR':t1_SR,'t1_str':t1_str,'t1_count':t1_count,
                            't2_GPA':t2_GPA,'t2_SR':t2_SR,'t2_str':t2_str,'t2_count':t2_count,
                            't3_GPA':t3_GPA,'t3_SR':t3_SR,'t3_str':t3_str,'t3_count':t3_count,
                            't4_GPA':t4_GPA,'t4_SR':t4_SR,'t4_str':t4_str,'t4_count':t4_count,
                            't5_GPA':t5_GPA,'t5_SR':t5_SR,'t5_str':t5_str,'t5_count':t5_count,
                            't6_GPA':t6_GPA,'t6_SR':t6_SR,'t6_str':t6_str,'t6_count':t6_count,
                            't7_GPA':t7_GPA,'t7_SR':t7_SR,'t7_str':t7_str,'t7_count':t7_count,
                            't8_GPA':t8_GPA,'t8_SR':t8_SR,'t8_str':t8_str,'t8_count':t8_count,
                            't9_GPA':t9_GPA,'t9_SR':t9_SR,'t9_str':t9_str,'t9_count':t9_count,
                            't10_GPA':t10_GPA,'t10_SR':t10_SR,'t10_str':t10_str,'t10_count':t10_count}, index=[1])
    out2 = pd.concat([stu_pid,out2])

C:\Users\qf3636ll\AppData\Local\Temp\ipykernel_23588\3322782570.py:137: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out2 = pd.concat([stu_pid,out2])


In [9]:
t1 = stats.ttest_ind(out2['SR'], out2['t1_SR'])
t2 = stats.ttest_rel(out2['SR'], out2['t1_SR'])

In [ ]:
listy3 = [1,2,3,4,5,6,7,8,9,10]
for i in listy3:
    testcol = f't{i}_GPA'
    t1 = stats.ttest_rel(out2['GPA'], out2[testcol])
    print(testcol,t1)

In [ ]:
out.to_csv('tests.csv',index=False)

In [ ]:
##separate the Learning Lab drops from the persists
pers = registrations.loc[(registrations['DROP_TIME_STAMP'].isna() == True) & (registrations['GRADE'].isna() == True),:]
drops = registrations.loc[(registrations['DROP_TIME_STAMP'].isna() == False) | (registrations['GRADE'].isna() == False),:]